In [76]:
import escher
from escher import Builder
import cobra
from time import sleep

In [77]:
escher.rc['never_ask_before_quit'] = True

builder

In [78]:

builder = Builder(
    model_json='iAB_RBC_283.json'
)
builder
    

Builder(never_ask_before_quit=True)

In [79]:
escher.list_available_maps()

[{'organism': 'Saccharomyces cerevisiae',
  'map_name': 'iMM904.Central carbon metabolism'},
 {'organism': 'Homo sapiens',
  'map_name': 'RECON1.Inositol retinol metabolism'},
 {'organism': 'Homo sapiens', 'map_name': 'RECON1.Glycolysis TCA PPP'},
 {'organism': 'Homo sapiens', 'map_name': 'RECON1.Tryptophan metabolism'},
 {'organism': 'Homo sapiens', 'map_name': 'RECON1.Carbohydrate metabolism'},
 {'organism': 'Homo sapiens',
  'map_name': 'RECON1.Amino acid metabolism (partial)'},
 {'organism': 'Escherichia coli', 'map_name': 'iJO1366.Nucleotide metabolism'},
 {'organism': 'Escherichia coli',
  'map_name': 'iJO1366.Fatty acid biosynthesis (saturated)'},
 {'organism': 'Escherichia coli',
  'map_name': 'iJO1366.Nucleotide and histidine biosynthesis'},
 {'organism': 'Escherichia coli', 'map_name': 'e_coli_core.Core metabolism'},
 {'organism': 'Escherichia coli', 'map_name': 'iJO1366.Central metabolism'},
 {'organism': 'Escherichia coli',
  'map_name': 'iJO1366.Fatty acid beta-oxidation'}

In [80]:
builder = Builder(
    map_name='e_coli_core.Core metabolism',
    model_name='e_coli_core',
)

In [81]:

builder

Builder(never_ask_before_quit=True)

In [82]:
# Load an Escher map
builder.map_name = 'iJO1366.Central metabolism'

In [83]:

# Load a COBRA model
builder.model_name = 'e_coli_core'

In [84]:

# Find any reactions that are in the map and not in the model, and turn them red
builder.highlight_missing = True

In [85]:

builder.model = cobra.io.load_json_model('iJO1366.json')

In [86]:
solution = builder.model.optimize()
builder.reaction_data = solution.fluxes

# Add some data for metabolites
builder.metabolite_data = solution.shadow_prices


# Add some data for metabolites
builder.metabolite_data = solution.shadow_prices

builder.reaction_scale = [
    { 'type': 'min', 'color': '#000000', 'size': 12 },
    { 'type': 'median', 'color': '#ffffff', 'size': 20 },
    { 'type': 'max', 'color': '#ff0000', 'size': 25 }
]

builder.reaction_scale_preset = 'GaBuRd'


# Make all the arrows three times as thick
builder.reaction_scale = [
    {k: v * 3 if k == 'size' else v for k, v in x.items()}
    for x in builder.reaction_scale
]

In [87]:
builder = Builder(
    height=600,
    map_name=None,
    model_name='e_coli_core',
    map_json='https://escher.github.io/1-0-0/5/maps/Escherichia%20coli/e_coli_core.Core%20metabolism.json',
)
builder

Builder(height=600, never_ask_before_quit=True)

In [88]:
model = cobra.io.load_json_model('e_coli_core.json')

In [89]:
reverse = False
step = 0.1
timestep = 0.1
duration = 15 # seconds
lim = [0, 0.5]
val = lim[-1]
for _ in range(int(duration / timestep)):
    model.reactions.EX_o2_e.lower_bound = -val
    solution = model.optimize()
    builder.reaction_data = solution.fluxes
    if val <= lim[0]:
        reverse = True
    if val >= lim[1]:
        reverse = False
    if reverse:
        val += step
    else:
        val -= step
    sleep(timestep)

In [ ]:
builder.save_html('example_map.html')